In [1]:
import sys
import os
sys.path.insert(0, os.path.abspath('..'))

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import time

plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('husl')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 11
%matplotlib inline

print("Diagnóstico de entorno:")
print(f"Python executable: {sys.executable}")
try:
    import cupy
    print(f"CuPy version importada directamente: {cupy.__version__}")
except Exception as e:
    print(f"Error al importar CuPy directamente: {e}")

print("-" * 50)

try:
    from src.ising_model_gpu import (
        run_mcmc_experiment_gpu,
        run_propp_wilson_experiment_gpu,
        GPU_AVAILABLE
    )
    
    if GPU_AVAILABLE:
        print("✅ GPU detectada y CuPy disponible desde el módulo.")
    else:
        print("⚠️ GPU no detectada desde el módulo.")
        import src.ising_model_gpu
        if hasattr(src.ising_model_gpu, '_IMPORT_ERROR'):
            print(f"Detalle del error capturado: {src.ising_model_gpu._IMPORT_ERROR}")
        else:
            print("No se encontró detalle del error.")
            
except ImportError:
    print("❌ Error importando el módulo src.ising_model_gpu.")

Diagnóstico de entorno:
Python executable: c:\Users\sergi\OneDrive - Universidad Nacional de Colombia\Documentos\proyectos\Cadenas_de_Markov\.venv\Scripts\python.exe
CuPy version importada directamente: 13.6.0
--------------------------------------------------
✅ GPU detectada y CuPy disponible desde el módulo.
CuPy version importada directamente: 13.6.0
--------------------------------------------------
✅ GPU detectada y CuPy disponible desde el módulo.


## Configuración de Experimentos

Usaremos los mismos parámetros que en la versión CPU, pero aprovechando la velocidad de la GPU para generar más muestras o hacerlo más rápido.

In [2]:
K = 12  # Tamaño del lattice
beta_values = np.arange(0, 1.1, 0.1)

# Parámetros para GPU (podemos ser más generosos con n_samples)
n_samples_mcmc = 1000
n_samples_pw = 100
max_time_pw = 1000000

## Ejecución MCMC (Gibbs Sampler) en GPU

In [3]:
mcmc_results = []

print(f"Iniciando MCMC en GPU para {len(beta_values)} valores de beta...")

for beta in tqdm(beta_values):
    res = run_mcmc_experiment_gpu(
        K=K, 
        beta=beta, 
        n_samples=n_samples_mcmc, 
        burn_in=1000, 
        thin=10 # En GPU corremos cadenas independientes, thin no es tan crítico para descorrelacionar
    )
    mcmc_results.append(res)

Iniciando MCMC en GPU para 11 valores de beta...


100%|██████████| 11/11 [00:07<00:00,  1.52it/s]


## Ejecución Propp-Wilson (CFTP) en GPU

In [ ]:
pw_results = []

print(f"Iniciando Propp-Wilson en GPU para {len(beta_values)} valores de beta...")

for beta in tqdm(beta_values):
    # Ajustar max_time para betas altos si es necesario
    current_max_time = max_time_pw
    if beta > 0.7:
        current_max_time = 5000000
        
    res = run_propp_wilson_experiment_gpu(
        K=K, 
        beta=beta, 
        n_samples=n_samples_pw, 
        max_time=current_max_time
    )
    pw_results.append(res)

Iniciando Propp-Wilson en GPU para 11 valores de beta...


 73%|███████▎  | 8/11 [3:13:31<2:26:24, 2928.05s/it]

Advertencia: 2 muestras no convergieron en T=5000000


 82%|████████▏ | 9/11 [6:09:06<2:44:57, 4948.89s/it]

Advertencia: 89 muestras no convergieron en T=5000000


## Análisis de Resultados

In [ ]:
df_mcmc = pd.DataFrame(mcmc_results)
df_pw = pd.DataFrame(pw_results)

plt.figure(figsize=(12, 6))
plt.errorbar(df_mcmc['beta'], df_mcmc['magnetization_mean'], 
             yerr=df_mcmc['magnetization_std'], fmt='o-', label='MCMC (GPU)', capsize=5)
plt.errorbar(df_pw['beta'], df_pw['magnetization_mean'], 
             yerr=df_pw['magnetization_std'], fmt='s--', label='Propp-Wilson (GPU)', capsize=5)

plt.title(f'Magnetización vs Beta (Lattice {K}x{K}) - GPU')
plt.xlabel('Beta (Temperatura Inversa)')
plt.ylabel('Magnetización Promedio')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(df_pw['beta'], df_pw['mean_coalescence_time'], 'o-', label='Tiempo Medio')
plt.plot(df_pw['beta'], df_pw['max_coalescence_time'], 's--', label='Tiempo Máximo')
plt.yscale('log')
plt.title('Tiempo de Coalescencia vs Beta (Propp-Wilson GPU)')
plt.xlabel('Beta')
plt.ylabel('Pasos de Tiempo (Log)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

In [ ]:
# Guardar resultados
os.makedirs('../resultados', exist_ok=True)
df_mcmc.to_csv('../resultados/resultados_mcmc_gpu.csv', index=False)
df_pw.to_csv('../resultados/resultados_pw_gpu.csv', index=False)
print("Resultados guardados en ../resultados/")